In [6]:
from graph_tool.all import Graph, graph_draw
import python_codes.files_operators
from deap import base, creator, tools, algorithms
import random

filename1 = "Germany50"

# 读取网络数据
read_graph, read_pos = python_codes.files_operators.read_files(f"../networks_clusters/{filename1}.net")
print("nodes: ", read_graph.num_vertices(), "links ", read_graph.num_edges())
graph_draw(read_graph, read_pos, vertex_text=read_graph.vertex_properties["number"], edge_color='blue', output_size=(1000, 1000), output=f'./output_GA/{filename1}.pdf')

# 确保所有边的权重都为1
weight = read_graph.new_edge_property("int")
for edge in read_graph.edges():
    weight[edge] = 1
read_graph.edge_properties["weight"] = weight

# 定义适应度函数
def eval_network(individual):
    distance = 0
    for edge_index in individual:
        edge = list(read_graph.edges())[edge_index]
        distance += 1  # 所有边的权重为1
    return (distance,)

# 定义个体和种群
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_edge", random.sample, range(read_graph.num_edges()), read_graph.num_edges())
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_edge)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 注册适应度函数、交叉、变异和选择操作
toolbox.register("evaluate", eval_network)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# 运行遗传算法
def main():
    random.seed(42)
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("min", min)
    stats.register("avg", lambda x: sum(x) / len(x))
    
    algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)
    
    return pop, stats, hof

if __name__ == "__main__":
    pop, stats, hof = main()
    best_individual = hof[0]
    print(f"Best individual: {best_individual}, Fitness: {best_individual.fitness.values[0]}")
    
    # 生成优化后的网络图
    optimized_graph = Graph(directed=False)
    optimized_pos = optimized_graph.new_vertex_property("vector<double>")
    vertex_map = {}
    
    number = optimized_graph.new_vertex_property("int")
    for v in read_graph.vertices():
        v_new = optimized_graph.add_vertex()
        vertex_map[int(v)] = v_new
        number[v_new] = read_graph.vp["number"][v]
        optimized_pos[v_new] = read_pos[v]
    
    optimized_graph.vertex_properties["number"] = number

    for edge_index in best_individual:
        edge = list(read_graph.edges())[edge_index]
        source, target = edge
        optimized_graph.add_edge(vertex_map[int(source)], vertex_map[int(target)])

    graph_draw(optimized_graph, optimized_pos, vertex_text=optimized_graph.vertex_properties["number"], edge_color='red', output_size=(1000, 1000), output=f'./output_GA/{filename1}_optimized.pdf')


nodes:  50 links  69
gen	nevals	min	avg
0  	100   	69 	69 
1  	85    	69 	69 
2  	76    	69 	69 
3  	74    	69 	69 
4  	80    	69 	69 
5  	80    	69 	69 
6  	76    	69 	69 
7  	76    	69 	69 
8  	83    	69 	69 
9  	70    	69 	69 
10 	77    	69 	69 
11 	80    	69 	69 
12 	73    	69 	69 
13 	80    	69 	69 
14 	80    	69 	69 
15 	63    	69 	69 
16 	78    	69 	69 
17 	81    	69 	69 
18 	79    	69 	69 
19 	76    	69 	69 
20 	67    	69 	69 
21 	82    	69 	69 
22 	70    	69 	69 
23 	77    	69 	69 
24 	84    	69 	69 
25 	67    	69 	69 
26 	79    	69 	69 
27 	69    	69 	69 
28 	62    	69 	69 
29 	76    	69 	69 
30 	71    	69 	69 
31 	69    	69 	69 
32 	80    	69 	69 
33 	76    	69 	69 
34 	79    	69 	69 
35 	81    	69 	69 
36 	81    	69 	69 
37 	82    	69 	69 
38 	77    	69 	69 
39 	76    	69 	69 
40 	77    	69 	69 
41 	76    	69 	69 
42 	80    	69 	69 
43 	70    	69 	69 
44 	72    	69 	69 
45 	84    	69 	69 
46 	84    	69 	69 
47 	81    	69 	69 
48 	81    	69 	69 
49 	82    	69 	69 
50 	74    